<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/wavelets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tsfresh
import pywt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.6 MB/s eta 0:00:00


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1ps_Bh93KQRJ4WbnilORmKd1BaWUXtmt1' # files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 400hz
Processing folder: 100hz
Processing folder: 800hz


In [3]:
import pandas as pd

In [4]:
train_target = pd.read_csv('100hz-train-target-20percent-1000files.csv')
train_target.drop(columns=['Unnamed: 0'], inplace = True)
test_target = pd.read_csv('100hz-test-target-20percent-1000files.csv')
test_target.drop(columns=['Unnamed: 0'], inplace = True)
train_feature = pd.read_csv('100hz-train-feature-20percent-1000files.csv')
train_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color'], inplace = True)
test_feature = pd.read_csv('100hz-test-feature-20percent-1000files.csv')
test_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color' ], inplace = True)

In [5]:
train_feature

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno
0,2024-08-14 19:17:58.406890:406,0.37,8.66,0.23,9.00,35.13,0
1,2024-08-14 19:17:58.414666:414,0.39,8.66,0.31,9.00,34.62,0
2,2024-08-14 19:17:58.422872:422,0.48,8.66,0.31,9.03,35.13,0
3,2024-08-14 19:17:58.435049:435,0.43,8.70,0.27,9.00,34.87,0
4,2024-08-14 19:17:58.443382:443,0.46,8.70,0.27,9.03,34.87,0
...,...,...,...,...,...,...,...
609718,2024-08-26 18:40:24.232560:232,0.37,8.75,0.31,9.03,32.87,3199
609719,2024-08-26 18:40:24.240992:240,0.33,8.70,0.31,9.03,32.87,3199
609720,2024-08-26 18:40:24.252801:252,0.31,8.75,0.23,9.03,32.62,3199
609721,2024-08-26 18:40:24.261297:261,0.37,8.75,0.31,9.00,32.62,3199


In [6]:
wavelet_type = 'db4'
decomp_level = 3

In [7]:
x_train_sorted = train_feature.sort_values(by=['fileno', 'Uhrzeit'])

# Group by fileno (each group is a time series)
grouped = x_train_sorted.groupby('fileno')

In [8]:
import numpy as np

In [9]:

# def tranform_ (column_):
#   wavelet_features = []
#   for fileno, group in grouped:
#       signal = group[f'{column_}'].values  # Example: Use power input

#       # Perform wavelet decomposition
#       coeffs = pywt.wavedec(signal, wavelet_type, level=decomp_level)

#       # Extract features from coefficients
#       features = {}
#       for i, coeff in enumerate(coeffs):
#           if len(coeff) == 0:
#               continue
#           # features: energy, entropy, variance
#           features[f'{column_}wavelet_energy_level_{i}'] = np.sum(coeff**2)
#           features[f'{column_}wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
#           features[f'{column_}wavelet_var_level_{i}'] = np.var(coeff)

#       # Add fileno to map features back
#       # features['fileno'] = fileno
#   wavelet_features.append(features)
#   return wavelet_features

# i_in = tranform_("I_In")
# i_out = tranform_("I_Out(A)")
# v_in = tranform_("V_In")
# v_out = tranform_("V_Out(V)")

# # Convert to DataFrame
# # pywt.wavet features['fileno'] = fileno

# # wavelet_df = pd.DataFrame(wavelet_features).set_index('fileno')

In [10]:
wavelet_df_I_In = {}
wavelet_df_I_Out = {}
wavelet_df_V_In = {}
wavelet_df_V_Out = {}
wavelet_df_Temp = {}

wavelet_features_I_In = []
wavelet_features_I_Out =[]
wavelet_features_V_In =[]
wavelet_features_V_Out =[]
wavelet_features_Temp =[]


for fileno, group in grouped:
    signal_I_In = group['I_In'].values
    signal_V_In = group['V_In'].values
    signal_I_Out = group['I_Out(A)'].values
    signal_V_Out = group['V_Out(V)'].values
    signal_Temp = group['Temp(°C)'].values

    # wavelet decomposition
    coeffs_I_In = pywt.wavedec(signal_I_In, wavelet_type, level=decomp_level)
    coeffs_V_In = pywt.wavedec(signal_V_In, wavelet_type, level=decomp_level)
    coeffs_I_Out = pywt.wavedec(signal_I_Out, wavelet_type, level=decomp_level)
    coeffs_V_Out = pywt.wavedec(signal_V_Out, wavelet_type, level=decomp_level)
    coeffs_Temp = pywt.wavedec(signal_Temp, wavelet_type, level=decomp_level)

# for I_In

    # Extract features from coefficients
    features_I_In = {}
    for i, coeff in enumerate(coeffs_I_In):
        if len(coeff) == 0:
            continue
        # features: energy, entropy, variance
        features_I_In[f'I_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_I_In[f'I_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_I_In[f'I_In_wavelet_var_level_{i}'] = np.var(coeff)

    # Add fileno to map features back
    features_I_In['fileno'] = fileno
    wavelet_features_I_In.append(features_I_In)


#for I_Out
    features_I_Out = {}
    for i, coeff in enumerate(coeffs_I_Out):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_I_Out[f'I_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_I_Out[f'I_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_I_Out[f'I_Out_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_I_Out.append(features_I_Out)



#for V_In
    features_V_In = {}
    for i, coeff in enumerate(coeffs_V_In):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_V_In[f'V_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_V_In[f'V_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_V_In[f'V_In_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_V_In.append(features_V_In)


#for V_Out
    features_V_Out = {}
    for i, coeff in enumerate(coeffs_V_Out):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_V_Out[f'V_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_V_Out[f'V_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_V_Out[f'V_Out_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_V_Out.append(features_V_Out)

#for Temp
    features_Temp = {}
    for i, coeff in enumerate(coeffs_Temp):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_Temp[f'Temp_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_Temp[f'Temp_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_Temp[f'Temp_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_Temp.append(features_Temp)

# Convert to DataFrame
wavelet_df_I_In = pd.DataFrame(wavelet_features_I_In)
wavelet_df_I_Out = pd.DataFrame(wavelet_features_I_Out)
wavelet_df_V_In = pd.DataFrame(wavelet_features_V_In)
wavelet_df_V_Out = pd.DataFrame(wavelet_features_V_Out)
wavelet_df_Temp = pd.DataFrame(wavelet_features_Temp)


wavelet_df_train = pd.concat([wavelet_df_I_In, wavelet_df_I_Out, wavelet_df_V_In, wavelet_df_V_Out, wavelet_df_Temp], axis=1)
wavelet_df_train = wavelet_df_train.set_index('fileno')


In [11]:
wavelet_df_train.isna().sum()

,0
I_In_wavelet_energy_level_0,0
I_In_wavelet_entropy_level_0,0
I_In_wavelet_var_level_0,0
I_In_wavelet_energy_level_1,0
I_In_wavelet_entropy_level_1,0
I_In_wavelet_var_level_1,0
I_In_wavelet_energy_level_2,0
I_In_wavelet_entropy_level_2,0
I_In_wavelet_var_level_2,0
I_In_wavelet_energy_level_3,0


In [12]:
x_test_sorted = test_feature.sort_values(by=['fileno', 'Uhrzeit'])

# Group by fileno (each group is a time series)
grouped = x_test_sorted.groupby('fileno')

In [13]:
wavelet_df_I_In = {}
wavelet_df_I_Out = {}
wavelet_df_V_In = {}
wavelet_df_V_Out = {}
wavelet_df_Temp = {}

wavelet_features_I_In = []
wavelet_features_I_Out =[]
wavelet_features_V_In =[]
wavelet_features_V_Out =[]
wavelet_features_Temp =[]


for fileno, group in grouped:
    signal_I_In = group['I_In'].values
    signal_V_In = group['V_In'].values
    signal_I_Out = group['I_Out(A)'].values
    signal_V_Out = group['V_Out(V)'].values
    signal_Temp = group['Temp(°C)'].values

    # wavelet decomposition
    coeffs_I_In = pywt.wavedec(signal_I_In, wavelet_type, level=decomp_level)
    coeffs_V_In = pywt.wavedec(signal_V_In, wavelet_type, level=decomp_level)
    coeffs_I_Out = pywt.wavedec(signal_I_Out, wavelet_type, level=decomp_level)
    coeffs_V_Out = pywt.wavedec(signal_V_Out, wavelet_type, level=decomp_level)
    coeffs_Temp = pywt.wavedec(signal_Temp, wavelet_type, level=decomp_level)

# for I_In

    # Extract features from coefficients
    features_I_In = {}
    for i, coeff in enumerate(coeffs_I_In):
        if len(coeff) == 0:
            continue
        # features: energy, entropy, variance
        features_I_In[f'I_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_I_In[f'I_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_I_In[f'I_In_wavelet_var_level_{i}'] = np.var(coeff)

    # Add fileno to map features back
    features_I_In['fileno'] = fileno
    wavelet_features_I_In.append(features_I_In)


#for I_Out
    features_I_Out = {}
    for i, coeff in enumerate(coeffs_I_Out):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_I_Out[f'I_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_I_Out[f'I_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_I_Out[f'I_Out_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_I_Out.append(features_I_Out)



#for V_In
    features_V_In = {}
    for i, coeff in enumerate(coeffs_V_In):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_V_In[f'V_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_V_In[f'V_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_V_In[f'V_In_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_V_In.append(features_V_In)


#for V_Out
    features_V_Out = {}
    for i, coeff in enumerate(coeffs_V_Out):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_V_Out[f'V_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_V_Out[f'V_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_V_Out[f'V_Out_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_V_Out.append(features_V_Out)

#for Temp
    features_Temp = {}
    for i, coeff in enumerate(coeffs_Temp):
        if len(coeff) == 0:
            continue
        # features: energy, entropy,
        features_Temp[f'Temp_wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features_Temp[f'Temp_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
        features_Temp[f'Temp_wavelet_var_level_{i}'] = np.var(coeff)

    wavelet_features_Temp.append(features_Temp)

# Convert to DataFrame
wavelet_df_I_In = pd.DataFrame(wavelet_features_I_In)
wavelet_df_I_Out = pd.DataFrame(wavelet_features_I_Out)
wavelet_df_V_In = pd.DataFrame(wavelet_features_V_In)
wavelet_df_V_Out = pd.DataFrame(wavelet_features_V_Out)
wavelet_df_Temp = pd.DataFrame(wavelet_features_Temp)


wavelet_df_test = pd.concat([wavelet_df_I_In, wavelet_df_I_Out, wavelet_df_V_In, wavelet_df_V_Out, wavelet_df_Temp], axis=1)
wavelet_df_test = wavelet_df_test.set_index('fileno')

In [14]:
wavelet_df_test.isna().sum()

,0
I_In_wavelet_energy_level_0,0
I_In_wavelet_entropy_level_0,0
I_In_wavelet_var_level_0,0
I_In_wavelet_energy_level_1,0
I_In_wavelet_entropy_level_1,0
I_In_wavelet_var_level_1,0
I_In_wavelet_energy_level_2,0
I_In_wavelet_entropy_level_2,0
I_In_wavelet_var_level_2,0
I_In_wavelet_energy_level_3,0


In [15]:
wavelet_df_test

,I_In_wavelet_energy_level_0,I_In_wavelet_entropy_level_0,I_In_wavelet_var_level_0,I_In_wavelet_energy_level_1,I_In_wavelet_entropy_level_1,I_In_wavelet_var_level_1,I_In_wavelet_energy_level_2,I_In_wavelet_entropy_level_2,I_In_wavelet_var_level_2,I_In_wavelet_energy_level_3,I_In_wavelet_entropy_level_3,I_In_wavelet_var_level_3,I_Out_wavelet_energy_level_0,I_Out_wavelet_entropy_level_0,I_Out_wavelet_var_level_0,I_Out_wavelet_energy_level_1,I_Out_wavelet_entropy_level_1,I_Out_wavelet_var_level_1,I_Out_wavelet_energy_level_2,I_Out_wavelet_entropy_level_2,I_Out_wavelet_var_level_2,I_Out_wavelet_energy_level_3,I_Out_wavelet_entropy_level_3,I_Out_wavelet_var_level_3,V_In_wavelet_energy_level_0,V_In_wavelet_entropy_level_0,V_In_wavelet_var_level_0,V_In_wavelet_energy_level_1,V_In_wavelet_entropy_level_1,V_In_wavelet_var_level_1,V_In_wavelet_energy_level_2,V_In_wavelet_entropy_level_2,V_In_wavelet_var_level_2,V_In_wavelet_energy_level_3,V_In_wavelet_entropy_level_3,V_In_wavelet_var_level_3,V_Out_wavelet_energy_level_0,V_Out_wavelet_entropy_level_0,V_Out_wavelet_var_level_0,V_Out_wavelet_energy_level_1,V_Out_wavelet_entropy_level_1,V_Out_wavelet_var_level_1,V_Out_wavelet_energy_level_2,V_Out_wavelet_entropy_level_2,V_Out_wavelet_var_level_2,V_Out_wavelet_energy_level_3,V_Out_wavelet_entropy_level_3,V_Out_wavelet_var_level_3,Temp_wavelet_energy_level_0,Temp_wavelet_entropy_level_0,Temp_wavelet_var_level_0,Temp_wavelet_energy_level_1,Temp_wavelet_entropy_level_1,Temp_wavelet_var_level_1,Temp_wavelet_energy_level_2,Temp_wavelet_entropy_level_2,Temp_wavelet_var_level_2,Temp_wavelet_energy_level_3,Temp_wavelet_entropy_level_3,Temp_wavelet_var_level_3
fileno,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200,106.187416,-31.843807,0.041383,0.334852,1.167507,0.003937,0.735930,1.936128,0.004466,0.472594,2.641604,0.001479,68.092076,7.744071,0.036460,0.423431,1.246498,0.004971,0.344618,1.821031,0.002112,0.692551,3.132306,0.002171,51041.260269,-326549.238730,0.003594,0.104018,0.623748,0.001224,0.288528,1.600078,0.001770,0.511026,2.918564,0.001591,55200.699008,-357484.173555,0.000298,0.022893,0.171208,0.000268,0.050046,0.371933,0.000307,0.095737,0.701449,0.000300,8.514145e+05,-7.843234e+06,0.008439,0.667193,2.728731,0.007842,1.812059,6.580892,0.010909,3.101068,10.562066,0.009710
201,95.984631,-31.967460,0.047518,0.271118,1.014867,0.003421,0.431211,1.850061,0.002965,0.855668,2.767745,0.003034,60.010670,6.349863,0.040641,0.329234,1.170894,0.004386,0.422259,1.896661,0.002921,0.501533,2.285820,0.001781,45066.944730,-288357.399845,0.002201,0.130874,0.724844,0.001745,0.189299,1.131002,0.001301,0.466424,2.595964,0.001632,48710.287766,-315455.542503,0.000339,0.015960,0.124061,0.000213,0.033600,0.257894,0.000233,0.079549,0.593236,0.000280,7.525325e+05,-6.933620e+06,0.017103,1.025061,3.539198,0.013353,2.158993,7.362270,0.014993,3.642101,11.924985,0.012953
202,101.636091,-28.848965,0.037623,0.742504,1.092157,0.008937,0.469536,1.829217,0.002936,0.449867,2.533407,0.001462,66.956693,5.451815,0.042611,0.477752,1.202873,0.005826,0.377228,1.916890,0.002401,0.699144,3.082516,0.002277,49201.970381,-314743.975686,0.002818,0.123483,0.684589,0.001496,0.247745,1.407760,0.001569,0.518031,2.963369,0.001651,53230.292049,-344701.516374,0.000294,0.016409,0.127502,0.000200,0.043089,0.325311,0.000274,0.089281,0.659897,0.000289,8.220838e+05,-7.573761e+06,0.018963,1.047892,3.695437,0.012775,2.019078,7.211774,0.012779,3.304980,11.213239,0.010743
203,106.779900,-33.392621,0.039327,0.430672,1.344045,0.005116,0.816126,1.909545,0.005040,0.582476,2.845195,0.001843,66.916355,7.923824,0.036266,0.344220,1.385569,0.004083,0.735201,2.281887,0.004566,0.667388,3.074074,0.002111,50458.921418,-322841.532686,0.002419,0.156101,0.876068,0.001844,0.270960,1.510916,0.001670,0.553161,3.118298,0.001735,54559.891390,-353342.855138,0.000240,0.024146,0.180630,0.000281,0.048212,0.358359,0.000298,0.085628,0.647538,0.000270,8.413951e+05,-7.750932e+06,0.007242,0.543158,2.354781,0.006145,1.250243

In [16]:
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

def xgb_(X, y, num_folds=5):

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=24)

    f1_scores = []
    accuracies = []
    precisions = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Initialize and train the XGBoost model with regularization
        xgb1 = xgb.XGBClassifier(
            objective="multi:softmax",
            random_state=42,
            alpha=0.1,
            reg_lambda=0.1,
            max_depth=4,
            min_child_weight=10,
            subsample=0.8,
            colsample_bytree=0.8
        )

        # Fit the model
        xgb1.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

        # Make predictions
        y_pred = xgb1.predict(X_test.to_numpy())

        # Ensure y_test is 1D for comparison
        y_test = y_test.to_numpy().ravel().astype(int)
        y_pred = y_pred.astype(int)

        # Calculate and store F1-score for this fold
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)

        # Calculate and store accuracy and precision for this fold
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred, average='weighted')
        precisions.append(precision)

        print(classification_report(y_test, y_pred))
        print(f"Fold {fold_idx + 1} F1-score: {f1:.4f}")
        print(f"Fold {fold_idx + 1} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold_idx + 1} Precision: {precision:.4f}")

        # Calculate and print confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

    # Calculate and print average metrics across all folds
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)

    print(f"Average F1-score across {num_folds} folds: {avg_f1:.4f}")
    print(f"Average Accuracy across {num_folds} folds: {avg_accuracy:.4f}")
    print(f"Average Precision across {num_folds} folds: {avg_precision:.4f}")

    # Final model: Train on the entire dataset
    print("Training final model on the entire dataset...")
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)

    # Final model with regularization
    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        random_state=42,
        alpha=1,
        reg_lambda=1,
        # max_depth=5,
        min_child_weight=20,
        # subsample=0.8,
        # colsample_bytree=0.8
    )

    # Train on the entire dataset
    final_model.fit(X_train_final.to_numpy(), y_train_final.to_numpy().ravel())

    # Evaluate the final model on the test set
    y_pred_final = final_model.predict(X_test_final.to_numpy())

    # Calculate accuracy, precision, confusion matrix for final model
    final_accuracy = accuracy_score(y_test_final, y_pred_final)
    final_precision = precision_score(y_test_final, y_pred_final, average='weighted')
    final_cm = confusion_matrix(y_test_final, y_pred_final)

    print(f"Final Model Accuracy: {final_accuracy:.4f}")
    print(f"Final Model Precision: {final_precision:.4f}")
    print("Final Model Confusion Matrix:")
    print(final_cm)

    # Save the final model
    # with open('XGB_final_model.pkl', 'wb') as f:
    #     pickle.dump(final_model, f)
    # files.download('XGB_final_model.pkl')



In [17]:
xgb_ (pd.concat([wavelet_df_train, wavelet_df_test]), pd.concat([train_target, test_target]))

Fold 1:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       210
           1       0.94      0.96      0.95       194
           2       0.95      0.95      0.95       199
           3       0.99      0.99      0.99       197

    accuracy                           0.97       800
   macro avg       0.97      0.97      0.97       800
weighted avg       0.97      0.97      0.97       800

Fold 1 F1-score: 0.9713
Fold 1 Accuracy: 0.9712
Fold 1 Precision: 0.9715
Confusion Matrix:
[[206   0   4   0]
 [  0 187   6   1]
 [  1   9 189   0]
 [  0   2   0 195]]
------------------------------
Fold 2:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       201
           1       0.93      0.92      0.93       191
           2       0.93      0.93      0.93       208
           3       0.97      0.98      0.98       200

    accuracy                           0.95       800
   macro avg       0.95 

In [18]:
train_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
795,3
796,3
797,3
798,3


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score

In [25]:
import sklearn

In [26]:
model2 = RandomForestClassifier()
model2.fit(wavelet_df_train, train_target)
y_pred = model2.predict(wavelet_df_test)
accura = sklearn.metrics.accuracy_score(test_target, y_pred)
print(accura)
precision_score = sklearn.metrics.precision_score(test_target, y_pred, average='weighted')
print(precision_score, " weighted")
print(classification_report(test_target, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.764375
0.7700556561585041  weighted
              precision    recall  f1-score   support

           0       0.98      0.90      0.93       800
           1       0.64      0.67      0.66       800
           2       0.73      0.82      0.77       800
           3       0.73      0.67      0.70       800

    accuracy                           0.76      3200
   macro avg       0.77      0.76      0.77      3200
weighted avg       0.77      0.76      0.77      3200



In [40]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score

final_model = xgb.XGBClassifier(
    # objective="multi:softmax",
    # random_state=42,
    alpha=10,
    reg_lambda=1,
    # max_depth=10,
    min_child_weight=10,
    # subsample=0.8,
    colsample_bytree=0.8
)

final_model.fit(wavelet_df_train, train_target)

y_pred_final = final_model.predict(wavelet_df_test)

final_accuracy = accuracy_score(test_target, y_pred_final)
final_precision = precision_score(test_target, y_pred_final, average='weighted')
final_cm = confusion_matrix(test_target, y_pred_final)

print(f"Final Model Accuracy: {final_accuracy:.4f}")
print(f"Final Model Precision: {final_precision:.4f}")
print("Final Model Confusion Matrix:")
print(final_cm)

Final Model Accuracy: 0.8000
Final Model Precision: 0.8128
Final Model Confusion Matrix:
[[718   0  80   2]
 [  0 585 130  85]
 [ 14  47 732   7]
 [  0 275   0 525]]
